# 스태킹 앙상블 (Stacking Ensemble)

---

개별 알고리즘으로 예측한 데이터를 기반으로 다시 예측을 수행

![stacking](./stacking.png)

In [1]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

C:\Users\river\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
cancer_data=load_breast_cancer()

X=cancer_data.data
y=cancer_data.target

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state=0)

In [3]:
knn_clf=KNeighborsClassifier(n_neighbors=4)
rf_clf=RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf=DecisionTreeClassifier()
ada_clf=AdaBoostClassifier(n_estimators=100)

# final model

lr_final=LogisticRegression(C=10)

In [4]:
# fitting

knn_clf.fit(x_train, y_train)
rf_clf.fit(x_train, y_train)
dt_clf.fit(x_train, y_train)
ada_clf.fit(x_train,y_train)

AdaBoostClassifier(n_estimators=100)

In [5]:
# predicted value

knn_pred=knn_clf.predict(x_test)
rf_pred=rf_clf.predict(x_test)
dt_pred=dt_clf.predict(x_test)
ada_pred=ada_clf.predict(x_test)

In [7]:
print('KNN Accuracy : {0:.4f}'.format(accuracy_score(y_test, knn_pred)))
print('Random Forest Accuracy : {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('Decision Tree Accuracy : {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('AdaBoost Accuracy : {0:.4f}'.format(accuracy_score(y_test, ada_pred)))

KNN Accuracy : 0.9211
Random Forest Accuracy : 0.9649
Decision Tree Accuracy : 0.9123
AdaBoost Accuracy : 0.9561


예측한 모델의 값들을 이어붙혀서 새로운 학습 세트를 만든다

In [10]:
pred_stack=np.c_[knn_pred, rf_pred, dt_pred, ada_pred]
pred_stack.shape

(114, 4)

In [11]:
lr_final.fit(pred_stack, y_test)

final_pred=lr_final.predict(pred_stack)

print("Final meta model's accuracy score : {0:.4f}".format(accuracy_score(y_test, final_pred)))

Final meta model's accuracy score : 0.9737


### 과적합 개선을 위한 CV 세트 기반의 스태킹 앙상블 (이해가 안됨..)

![cv_stacking](./cv_stacking.png)

In [18]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수
def get_stacking_base_datasets(model, x_train_n, y_train_n, x_test_n, n_folds):
    kf=KFold(n_splits=n_folds, shuffle=False)
    
    train_fold_pred=np.zeros((x_train_n.shape[0],1))
    test_pred=np.zeros((x_test_n.shape[0],n_folds))
    print(model.__class__.__name__, ' model 시작')
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(x_train_n)):
        # 입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
        print('\t 폴드 세트 : ', folder_counter, ' 시작')
        x_tr=x_train_n[train_index]
        y_tr=y_train_n[train_index]
        x_te=x_train_n[valid_index]
        
        model.fit(x_tr, y_tr)
        
        train_fold_pred[valid_index, :]=model.predict(x_te).reshape(-1,1)
        
        test_pred[:, folder_counter]=model.predict(x_test_n)
        
    test_pred_mean=np.mean(test_pred, axis=1).reshape(-1,1)
    
    return train_fold_pred, test_pred_mean

In [20]:
knn_train ,knn_test=get_stacking_base_datasets(knn_clf, x_train, y_train, x_test, 7)
rf_train, rf_test=get_stacking_base_datasets(rf_clf, x_train, y_train, x_test, 7)
dt_train, dt_test=get_stacking_base_datasets(dt_clf, x_train, y_train, x_test, 7)
ada_train, ada_test=get_stacking_base_datasets(ada_clf, x_train, y_train, x_test, 7)

KNeighborsClassifier  model 시작
	 폴드 세트 :  0  시작
	 폴드 세트 :  1  시작
	 폴드 세트 :  2  시작
	 폴드 세트 :  3  시작
	 폴드 세트 :  4  시작
	 폴드 세트 :  5  시작
	 폴드 세트 :  6  시작
RandomForestClassifier  model 시작
	 폴드 세트 :  0  시작
	 폴드 세트 :  1  시작
	 폴드 세트 :  2  시작
	 폴드 세트 :  3  시작
	 폴드 세트 :  4  시작
	 폴드 세트 :  5  시작
	 폴드 세트 :  6  시작
DecisionTreeClassifier  model 시작
	 폴드 세트 :  0  시작
	 폴드 세트 :  1  시작
	 폴드 세트 :  2  시작
	 폴드 세트 :  3  시작
	 폴드 세트 :  4  시작
	 폴드 세트 :  5  시작
	 폴드 세트 :  6  시작
AdaBoostClassifier  model 시작
	 폴드 세트 :  0  시작
	 폴드 세트 :  1  시작
	 폴드 세트 :  2  시작
	 폴드 세트 :  3  시작
	 폴드 세트 :  4  시작
	 폴드 세트 :  5  시작
	 폴드 세트 :  6  시작


In [21]:
stacking_final_x_train=np.c_[knn_train, rf_train, dt_train, ada_train]
stacking_final_x_test=np.c_[knn_test, rf_test, dt_test, ada_test]

In [22]:
print('원본 학습 데이터 {0} , 테스트 데이터 {1}'.format(x_train.shape , x_test.shape))
print('스태킹 학습 데이터 {0} , 테스트 데이터 {1}'.format(stacking_final_x_train.shape, stacking_final_x_test.shape))

원본 학습 데이터 (455, 30) , 테스트 데이터 (114, 30)
스태킹 학습 데이터 (455, 4) , 테스트 데이터 (114, 4)


In [23]:
lr_final.fit(stacking_final_x_train, y_train)

stack_final=lr_final.predict(stacking_final_x_test)

print("Final Meta model's accuracy score : {0:.4f}".format(accuracy_score(y_test, stack_final)))

Final Meta model's accuracy score : 0.9737
